# Lecture 9: Subqueries in SQL

### Connecting to the Database


In [163]:
# load the ipython-sql extension
%load_ext sql

import json
import urllib.parse

with open('data/credentials.json') as f:
    login = json.load(f)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Let's connect to your dvdrental database

In [164]:
%sql postgresql://{username}:{password}@{host}:{port}/dvdrental

'Connected: postgres@dvdrental'